In [3]:
import urllib.request
from bs4 import BeautifulSoup
import re as re
import requests
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import time
import click

In [19]:
#All information is gathered from www.weather.com

headers = {
    'User-Agent': 'Weather bot',
    'From': 'dylansuther12@gmail.com'
}


#Page information for selenium
baseURL = 'https://www.weather.com'

#Chromedriver information for selenium
chrome_options = Options()
chrome_options.add_argument('--headless')
driver = webdriver.Chrome(options=chrome_options)
driver.get(baseURL)

def menu():
    welcome = "\nHi! Please choose weather data you would like to see!" 
    menuChoices = "\n1. Current Forecast\n2. Ten Day Forecast\n3. Hourly Forecast\n4. Enter a new location\n5. Exit\n"
    choice = input(welcome + "\n" + "-" * len(welcome) + menuChoices)
    return choice

def chooseAgain():
    contChoice = input(contString)
    if(contChoice == 'y' or contChoice == 'Y'):
        return True
    elif(contChoice == 'n' or contChoice == 'N'):
        print("\nThanks! Have a wonderful day!")
        return False
    else:
        print("\nInvalid option. Please select again")
        return chooseAgain()
    
    
    

#Search for address using selenium
def searchWeather(location): 
    
    soupURL = driver.current_url
    page = urllib.request.urlopen(soupURL).read()
    weatherSoup = BeautifulSoup(page)
    
    try:
        address = driver.find_element_by_id("LocationSearch_input")
        time.sleep(2)
        address.send_keys(location)
        time.sleep(2)
        address.send_keys(Keys.ENTER)
        
        

    except StaleElementReferenceException:
        address = driver.find_element_by_id("LocationSearch_input")
        time.sleep(2)
        address.send_keys(location)
        time.sleep(2)
        address.send_keys(Keys.ENTER)
        
#Print out weather forecast for today
def currentWeather():
    
    soupURL = driver.current_url
    page = urllib.request.urlopen(soupURL).read()
    weatherSoup = BeautifulSoup(page)
    
    severeWeather = weatherSoup.find(class_ = "AlertHeadline--alertText--38xov")
    
    if severeWeather is not None:
        driver.get(soupURL)
        button = driver.find_element_by_class_name("AlertHeadline--alertText--38xov")
        button.click()
        time.sleep(2)
        severeURL = driver.current_url
        severePage = urllib.request.urlopen(severeURL).read()
        severeSoup = BeautifulSoup(severePage)
        
        severeData = {'header' : severeSoup.find(class_ = "AlertHeadline--alertText--38xov AlertHeadline--hasSubheadline--1yIXE AlertDetailsCard--headlineText--yuQ2z").text,
                      'subheader' : severeSoup.find(class_ = "AlertHeadline--subheadlineText--36etT").text,
                      'affectedAreaHeading' : severeSoup.find(class_ = "AlertDetails--label--2sDib").text,
                      'affectedAreaDetails' : severeSoup.find(class_ = "AlertDetails--value--DiwUQ").text
            
        }
        
        severeString = (severeData['header'] + "\n" + severeData['subheader'] + "\n\n" + severeData['affectedAreaHeading']
             + "\n" + severeData['affectedAreaDetails'])
        print("\n" + "*" * len(severeData['affectedAreaDetails']))
        print("\n" + severeString + "\n")
        print("*" * len(severeData['affectedAreaDetails']))
        button = driver.find_element_by_class_name("styles--liContent--3lr9O")
        button.click()
        time.sleep(2)
            
        
            
            
            
    weatherData = {
        'location' : weatherSoup.find(class_ = "CurrentConditions--location--1YWj_").text,
        'humidity' : weatherSoup.find(class_ = "WeatherDetailsListItem--wxData--kK35q").text,
        'currentTemp': weatherSoup.find(class_ = "CurrentConditions--tempValue--MHmYY").text,
        'windSpeed' : weatherSoup.find(class_ = "Wind--windWrapper--3Ly7c undefined").text[14:],
        'currentConditions': weatherSoup.find(class_ = "CurrentConditions--phraseValue--mZC_p").text
    }
    
    print("\nHere is the current weather forecast for " + weatherData['location'] + "\n" + "-" * + (len("Here is the current weather data for " +
                                                                               weatherData['location']) + 4))
    print("Current Temperature: " + weatherData['currentTemp'] + "\nCurrent Wind Speed: " +
          weatherData['windSpeed'] + "\nCurrent Humidity: " + weatherData['humidity'] +
          "\nCurrent Weather Conditions: " + weatherData['currentConditions'])

#Print out ten day weather forecast
def tendayWeather():
    soupURL = driver.current_url[:27] + "/tenday" + driver.current_url[33:]
    page = urllib.request.urlopen(soupURL).read()
    weatherSoup = BeautifulSoup(page)
    
    detailIndex = weatherSoup.find_all(id = re.compile("detailIndex"))
    
    location = weatherSoup.find(class_ = "LocationPageTitle--PresentationName--1AMA6").text
    
    print("\nHere is the ten day weather forecast for " + location + "\n" + "-" * + (len("Here is the ten day weather forecast for " +
                                                                               location) + 1))


    for i in range(0, 10):
        weatherData = {
            'date' : detailIndex[i].find(class_ = "DailyContent--daypartDate--3VGlz").text,
            'highTemp' : detailIndex[i].find(class_ = "DetailsSummary--highTempValue--3PjlX").text,
            'lowTemp' : detailIndex[i].find(class_ = "DetailsSummary--lowTempValue--2tesQ").text,
            'precipitation' : detailIndex[i].find(class_ = "DailyContent--value--1Jers").text,
            'windSpeed' : detailIndex[i].find(class_ = "Wind--windWrapper--3Ly7c DailyContent--value--1Jers").text.split(" ")[1] + " mph",
            'summary' : detailIndex[i].find(class_="DailyContent--narrative--3Ti6_").text
        }
        weatherString = ("\n"
        + click.style(weatherData['date'], bold=True) 
        + click.style("\n\nHigh Temperature: ", bold=True) 
        + weatherData['highTemp'] 
        + click.style(" F\nLow Temperature: ", bold=True) 
        + weatherData['lowTemp'] 
        + click.style(" F\nPrecipitation: ", bold=True) 
        + weatherData['precipitation'] 
        + click.style("\nWind Speed: ", bold=True) 
        + weatherData['windSpeed'] 
        + "\n\n")  

        box = "-" * (len(max(weatherString.split("\n"))) + 4)

        print(f"{box}\n{weatherString.center(len(box)-2)}\n{box}")
    
def hourlyWeather():
    soupURL = driver.current_url[:27] + "/hourbyhour" + driver.current_url[33:]
    page = urllib.request.urlopen(soupURL).read()
    weatherSoup = BeautifulSoup(page)
    
    location = weatherSoup.find(class_ = "LocationPageTitle--PresentationName--1AMA6").text
    detailSummary = weatherSoup.find_all(class_ = "DetailsSummary--DetailsSummary--1DqhO")

    print("\nHere is your hourly weather forecast for " + location + "\n" + "-" * 
         (len("Here is your hourly weather forecast for" + location) + 1))

    for i in range(0, 12):
        weatherData = {
            'time': detailSummary[i].find(class_ = "DetailsSummary--daypartName--kbngc").text,
            'temperature' : detailSummary[i].find(class_ = "DetailsSummary--tempValue--jEiXE").text,
            'currentConditions' : detailSummary[i].find(class_ = "DetailsSummary--extendedData--307Ax").text,
            'precipitation' : detailSummary[i].find(class_ = "DetailsSummary--precip--1a98O").find("span").text,
            'wind' : detailSummary[i].find(class_ = "Wind--windWrapper--3Ly7c undefined").text
        }

        weatherString = ("\n"
                        + "Time: "
                        + weatherData['time']
                        + "\nTemperature: "
                        + weatherData['temperature']
                        + "\nConditions: "
                        + weatherData['currentConditions']
                        + "\nPrecipitation: "
                        + weatherData['precipitation']
                        + "\nWind: "
                        + weatherData['wind']
                        + "\n\n")
        print(weatherString)
    
    
    
location = input("Enter a location to get weather information: ")
searchWeather(location)

menuFlag = True
contString = "\nWould you like to make another choice? (y/n)"

while(menuFlag):
    menuChoice = menu()
    if(menuChoice == '1'):
        currentWeather()
        menuFlag = chooseAgain()
    elif(menuChoice == '2'):
        tendayWeather()
        menuFlag = chooseAgain()
    elif(menuChoice == '3'):
        hourlyWeather()
        menuFlag = chooseAgain()
    elif(menuChoice == '4'):
        location = input("Enter a location to get weather information: ")
        searchWeather(location)
    elif(menuChoice == '5'):
        print("\nThanks! Have a wonderful day!")
        menuFlag = False
    else:
        print("\nInvalid option. Please select again")




Enter a location to get weather information: Bowling Green Ohio

Hi! Please choose weather data you would like to see!
------------------------------------------------------
1. Current Forecast
2. Ten Day Forecast
3. Hourly Forecast
4. Enter a new location
5. Exit
1

***********************************************

River Flood Warning
Until Wed 11 pm EST

Action Recommended
Avoid the subject event as per the instructions

***********************************************

Here is the current weather forecast for Bowling Green, OH
----------------------------------------------------------
Current Temperature: 38°
Current Wind Speed: 5 mph
Current Humidity: --/37°
Current Weather Conditions: Clear

Would you like to make another choice? (y/n)n

Thanks! Have a wonderful day!
